In [1]:
import requests as r
import pandas as pd
import os

import sys
import csv
from datetime import datetime, timedelta
import time
sys.path.append("../../helper_functions")
import clickhouse_utils as ch
sys.path.pop()

'../../helper_functions'

In [2]:
client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

end_date = datetime(2024, 6, 27)   # Year, Month, Day
trailing_days = 1

chain_names = [
        # 'op',
        'base']

In [3]:
query = '''
SELECT *
FROM @chain_name@_transactions
WHERE gas_price > 0
        AND block_timestamp >= '@start_date@'
        AND block_timestamp < '@end_date@'
        AND receipt_status = 1

INTO OUTFILE '@chain_name@_14_txs.csv.gz'
COMPRESSION 'gzip'
FORMAT CSVWithNames

SETTINGS max_execution_time = 500000

'''


In [4]:
# Calculate start date
start_date = end_date - timedelta(days=trailing_days)


In [5]:
for chain_name in chain_names:
    result = None
    start_time = time.time()
    print(f"Starting processing for {chain_name}")

    csv_filename = f'csv_outputs/{chain_name}_{end_date.strftime("%Y%m%d")}_{trailing_days}days.csv'
    file_exists = os.path.isfile(csv_filename)

    q_run = query.replace('@chain_name@', chain_name)
    q_run = q_run.replace('@start_date@', start_date.strftime('%Y-%m-%d'))
    q_run = q_run.replace('@end_date@', end_date.strftime('%Y-%m-%d'))

    result = client.query_df(q_run)
    # result.to_csv(csv_filename, index=False, mode='w')

    int_time = time.time()
    execution_time = int_time - start_time
    print(f"Query for {chain_name} completed in {execution_time:.2f} seconds")

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Processing for {chain_name} completed in {execution_time:.2f} seconds")
    print(f"Processed data for {trailing_days} days, ending on {end_date.date()}")
    print(f"Results saved to {csv_filename}")
    print("---")

Starting processing for base


DatabaseError: :HTTPDriver for https://pdmv9lhojy.us-west-2.aws.clickhouse.cloud:8443 returned response code 400)
 Code: 62. DB::Exception: Syntax error: failed at position 256 ('FORMAT') (line 15, col 2): FORMAT Native. Expected end of query. (SYNTAX_ERROR) (version 24.2.2.16370 (official build))


In [ ]:
# print(result.result_rows)